In [56]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

def make_json_serializable(obj):
    if isinstance(obj, dict):
        return {k: make_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_json_serializable(item) for item in obj]
    elif isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, tuple):
        return tuple(make_json_serializable(item) for item in obj)
    elif isinstance(obj, (str, int, float, bool, type(None))):
        return obj
    elif hasattr(obj, '__dict__'):
        return make_json_serializable(obj.__dict__)
    else:
        return str(obj)

# read file in and filter dataframe
file_path = 'hcmst2017to2022.csv'
df = pd.read_csv(file_path)

for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

columns_to_print_section = ['caseid_new', 'w1_section', 'w2_section', 'w3_section', 'w3_partner_source']
print("Number of records in dataset:", df[columns_to_print_section].shape)

Number of records in dataset: (3510, 5)


In [57]:
# all the people that were in one relationship throughout all w1->w3
con_w1_partnered = df['w1_section'] == 1 # those who were partenered in 2017
con_w2_married_same = df['w2_section'] == 1 # who are still married to the same partner in wave 2
con_w2_part_same = df['w2_section'] == 3 # or still partnered with that same partner but not married (but still together) in wave 2 
# (note that we are not looking at those with new partners in w2/w3)
con_w3_married_same = df['w3_section'] == 1 # who are still married to the same partner in wave 3
con_w3_partnered_same = df['w3_section'] == 3 # or still partnered with that same partner but not married (but still together) in wave 2 
con_w3_same_part_fromw1 = df['w3_partner_source'] == 1

com_con_same_rel_all_waves = con_w1_partnered & (con_w2_married_same | con_w2_part_same) & ((con_w3_married_same | con_w3_partnered_same) & con_w3_same_part_fromw1)
same_rel_all_waves_df = df.loc[com_con_same_rel_all_waves]
print(len(same_rel_all_waves_df))

1096


In [58]:
# now creating dict 3 for those who had same rel w1->w3
rel_counter = 0
dict_same_rel = {}

df_dict_same_rel = same_rel_all_waves_df.to_dict(orient='records')

for record in df_dict_same_rel:
    dict_same_rel[rel_counter] = {
        'caseid_new': record['caseid_new'], #resp case id
        'w1_ppage': record['w1_ppage'], #age
        'w1_q9': record['w1_q9'], #partner's age in 2017

        'w1_subject_race': record['w1_subject_race'], #resp race
        'w1_q6b': record['w1_q6b'], #partner's race
                
        'w1_ppeduc': record['w1_ppeduc'], #resp educ level
        'w1_q10': record['w1_q10'], #partner educ level

        'w1_partyid7':record['w1_partyid7'],
        'w1_q12':record['w1_q12'],
        'political_steps': np.abs(record['w1_partyid7']-record['w1_q12']),
            }
    rel_counter += 1

In [59]:
partner_info = {}
rec_counter = 1096

for record in df_dict_same_rel:
    partner_info [rec_counter] = {
        "r_caseid":record["caseid_new"],
        "r_gender": record["w1_ppgender"], # resp gender
        "p_gender": record["w1_q4"], # partner's gender
        
        "r_race": record["w1_ppethm"], # resp race
        "p_race": record["w1_q6b"], #partner's race
        
        "r_age": record["w1_ppage"], # resp age
        "p_age": record["w1_q9"], #partner age in 2017,
        
        "r_edu": record["w1_ppeduc"], # resp education
        "p_edu": record["w1_q10"], # partner education
        
        "r_politic": record["w1_partyid7"], # resp political affiliation
        "p_politic": record["w1_q12"] # partner political affiliation
        
    }
    rec_counter +=1
    
print("partner_info dict format:", partner_info[1096])

partner_info dict format: {'r_caseid': 71609, 'r_gender': 2, 'p_gender': 1.0, 'r_race': 1, 'p_race': 1.0, 'r_age': 68, 'p_age': 71.0, 'r_edu': 10, 'p_edu': 10.0, 'r_politic': 3.0, 'p_politic': 3.0}


In [60]:
# split couples into individual people and their attributes
all_people_id = set()
all_people = {}


for record in partner_info:
    id_1 = partner_info[record]["r_caseid"]
    id_2 = record

    all_people_id.add(id_1)
    all_people_id.add(id_2)

    all_people[id_1] = {
        "race": partner_info[record]["r_race"],
        "age": partner_info[record]["r_age"],
        "edu": partner_info[record]["r_edu"],
        "pol": partner_info[record]["r_politic"]
    }

    all_people[id_2] = {
        "race": partner_info[record]["p_race"],
        "age": partner_info[record]["p_age"],
        "edu": partner_info[record]["p_edu"],
        "pol": partner_info[record]["p_politic"]
    }

print("Number of Individual People: ", len(all_people))

Number of Individual People:  2192


In [61]:
# scoring algo from original algorithm
def cal_pol_steps(a,b):
    if (a>=1 and b>=1):
        pol_score= np.abs(np.abs(int(a)-int(b))-6)
        return round((pol_score/6)*4,3)
    else:
        return 0

def check_eth_same(a,b):
    same_eth=0
    if (a>=1 and b>=1):
        if (a==b):
            same_eth=1
            return round((same_eth/1)*3,3)
    return same_eth

def cal_age_diff(r_age,p_age):
    score_age=0
    if (r_age>=1 and p_age>=1):
        age_diff = np.abs(r_age-p_age)
        if (age_diff>=2 and age_diff<=5):
            score_age=1
            return round((score_age/1)*2,3)
    return score_age
    
def check_p_ed_greater(r_edu,p_edu):
    if (r_edu>=1 and p_edu>=1):    
        score_edu = np.abs(np.abs(r_edu-p_edu)-13)
        return round((score_edu/13)*1,3)
    else:
        return 0

In [62]:
# need to export in format that Python matching library can use
# link: https://pypi.org/project/matching/
# link 2: https://daffidwilde.github.io/matching/docs/reference/stable_roommates.html

# need: for each person x, a list that has every other person y in the group in order of person x's preference of dating them
# preference = score calculated with functions above ^
# for each person, will calculate a score for every other person (n^2) and make a sorted list

all_people_pref_scores = {}
all_people2 = all_people

i1 =0
i2=0

for record in all_people:
    id_1 = record
    score_pid = {}
    for record2 in all_people2:
        id_2 = record2
        p_score = cal_pol_steps(all_people[id_1]["pol"], all_people[id_2]["pol"])
        r_score = check_eth_same(all_people[id_1]["race"], all_people[id_2]["race"])
        a_score = cal_age_diff(all_people[id_1]["age"], all_people[id_2]["age"])
        e_score = check_p_ed_greater(all_people[id_1]["edu"], all_people[id_2]["edu"])
        score = p_score + r_score + a_score + e_score
        score_pid[id_2] = score
            
    score_pid_sorted = dict(sorted(score_pid.items(), key=lambda item: item[1], reverse=True))
    #print(score_pid)
    all_people_pref_scores[record] = score_pid_sorted
            
            

In [63]:
keys_score = list(all_people_pref_scores.keys())

In [64]:
print(all_people_pref_scores[71609])
print(len(all_people_pref_scores[71609]))

{1096: 10.0, 1155: 10.0, 1140491: 10.0, 1786749: 10.0, 1446: 10.0, 1468: 10.0, 1488: 10.0, 1497: 10.0, 1975919: 10.0, 2485815: 10.0, 2679531: 10.0, 2724951: 10.0, 2006: 10.0, 2882059: 10.0, 2145: 10.0, 214227: 9.923, 1103: 9.923, 1125: 9.923, 1180: 9.923, 1192: 9.923, 1304: 9.923, 1550311: 9.923, 1633263: 9.923, 1380: 9.923, 1925505: 9.923, 1569: 9.923, 2092107: 9.923, 1683: 9.923, 2311713: 9.923, 1813: 9.923, 1839: 9.923, 2449593: 9.923, 2493475: 9.923, 1895: 9.923, 1897: 9.923, 1956: 9.923, 1970: 9.923, 1975: 9.923, 2015: 9.923, 2035: 9.923, 2846681: 9.923, 981251: 9.846, 1348: 9.846, 1768829: 9.846, 2136629: 9.846, 1758: 9.846, 1886: 9.846, 2714381: 9.846, 2763005: 9.846, 2138: 9.846, 2166: 9.846, 1049013: 9.769, 1214: 9.769, 1966321: 9.769, 2854889: 9.769, 2914073: 9.769, 1826: 9.692, 1119: 9.333, 1333997: 9.333, 1288: 9.333, 1374661: 9.333, 1445543: 9.333, 1487: 9.333, 1910803: 9.333, 1550: 9.333, 1557: 9.333, 1749: 9.333, 1846: 9.333, 2591515: 9.333, 1991: 9.333, 2038: 9.333, 207

In [65]:
# need to remove the score for self (each person should only have 2191)
for record in all_people_pref_scores:
    preferences = all_people_pref_scores[record]
    to_remove = []
    for record2 in preferences:
        if record == record2:
            #print("R1:", record)
            #print("R2:", record2)
            preferences.pop(record2, None)
            break

        

In [66]:
print(len(all_people_pref_scores[1096]))

2191


In [67]:
print("Format of all_pref_scores..... key = 71609 : ", all_people_pref_scores[71609])

Format of all_pref_scores..... key = 71609 :  {1096: 10.0, 1155: 10.0, 1140491: 10.0, 1786749: 10.0, 1446: 10.0, 1468: 10.0, 1488: 10.0, 1497: 10.0, 1975919: 10.0, 2485815: 10.0, 2679531: 10.0, 2724951: 10.0, 2006: 10.0, 2882059: 10.0, 2145: 10.0, 214227: 9.923, 1103: 9.923, 1125: 9.923, 1180: 9.923, 1192: 9.923, 1304: 9.923, 1550311: 9.923, 1633263: 9.923, 1380: 9.923, 1925505: 9.923, 1569: 9.923, 2092107: 9.923, 1683: 9.923, 2311713: 9.923, 1813: 9.923, 1839: 9.923, 2449593: 9.923, 2493475: 9.923, 1895: 9.923, 1897: 9.923, 1956: 9.923, 1970: 9.923, 1975: 9.923, 2015: 9.923, 2035: 9.923, 2846681: 9.923, 981251: 9.846, 1348: 9.846, 1768829: 9.846, 2136629: 9.846, 1758: 9.846, 1886: 9.846, 2714381: 9.846, 2763005: 9.846, 2138: 9.846, 2166: 9.846, 1049013: 9.769, 1214: 9.769, 1966321: 9.769, 2854889: 9.769, 2914073: 9.769, 1826: 9.692, 1119: 9.333, 1333997: 9.333, 1288: 9.333, 1374661: 9.333, 1445543: 9.333, 1487: 9.333, 1910803: 9.333, 1550: 9.333, 1557: 9.333, 1749: 9.333, 1846: 9.333,

In [68]:
# Convert dictionary to JSON-serializable format
all_people_pref_score_json = make_json_serializable(all_people_pref_scores)

with open("pref_scores.json", "w") as json_file:
    json.dump(all_people_pref_score_json, json_file, indent=4)

In [69]:
# matches from matching library run on collab;
match_data = {
   71609: 1096,
   1096: 71609,
   106983: 1903,
   1097: 1177,
   164061: 1170887,
   1098: 822315,
   212249: 1648699,
   1099: 1692049,
   214227: 646023,
   1100: 1587583,
   218351: 2071877,
   1101: 2602801,
   220655: 1203831,
   1102: 2144713,
   291177: 1697675,
   1103: 842437,
   369975: 855297,
   1104: 2283991,
   428211: 1413,
   1105: 1343,
   497203: 1130,
   1106: 1815129,
   516823: 810569,
   1107: 2228393,
   582849: 1168,
   1108: 1129,
   587125: 1183,
   1109: 1593839,
   589881: 1186,
   1110: 981685,
   608697: 1283019,
   1111: 1011851,
   621641: 1692681,
   1112: 1121,
   632253: 1884543,
   1113: 2117,
   634833: 1494,
   1114: 1478,
   637531: 1809325,
   1115: 1442,
   643423: 1155,
   1116: 1571537,
   643505: 2142099,
   1117: 1139,
   646023: 214227,
   1118: 1216,
   646157: 1276181,
   1119: 1557,
   648669: 703017,
   1120: 1272909,
   650237: 2770541,
   1121: 1112,
   651259: 1177155,
   1122: 1307,
   651349: 1134,
   1123: 1097625,
   653633: 1374091,
   1124: 2961203,
   654589: 2170187,
   1125: 1188,
   657869: 1416,
   1126: 1195,
   662355: 1161211,
   1127: 817367,
   694739: 1128,
   1128: 694739,
   698117: 1439945,
   1129: 1108,
   703017: 648669,
   1130: 497203,
   704661: 1473855,
   1131: 1223,
   705457: 1132,
   1132: 705457,
   709075: 1342,
   1133: 1141,
   709173: 2573527,
   1134: 651349,
   724199: 2056,
   1135: 774843,
   726183: 1073943,
   1136: 1234,
   726431: 1190,
   1137: 1119259,
   727775: 1712517,
   1138: 1354,
   728327: 1277,
   1139: 1117,
   729319: 2781391,
   1140: 1267,
   730391: 1206,
   1141: 1133,
   731249: 1269,
   1142: 1146,
   731629: 1001637,
   1143: 1221467,
   734689: 1144,
   1144: 734689,
   755761: 2015057,
   1145: 1163,
   769207: 1323365,
   1146: 1142,
   774843: 1135,
   1147: 2613043,
   775045: 1966851,
   1148: 1662,
   782789: 1256,
   1149: 1738505,
   783399: 1236,
   1150: 1263,
   783679: 1768829,
   1151: 1360,
   784141: 1123417,
   1152: 1189,
   792153: 1979935,
   1153: 1374,
   799683: 1171,
   1154: 2327101,
   806423: 1237,
   1155: 643423,
   810569: 516823,
   1156: 1282,
   811175: 1418329,
   1157: 1257,
   812769: 1158,
   1158: 812769,
   813467: 1927273,
   1159: 1196757,
   814285: 2168,
   1160: 2109,
   815831: 1101051,
   1161: 1536983,
   817367: 1127,
   1162: 2152283,
   821909: 1130879,
   1163: 1145,
   822315: 1098,
   1164: 1381,
   842437: 1103,
   1165: 1765905,
   844189: 1607815,
   1166: 1287847,
   844865: 1248,
   1167: 1780607,
   846827: 1828311,
   1168: 582849,
   854869: 1295,
   1169: 1904,
   855297: 369975,
   1170: 1501,
   859445: 2017641,
   1171: 799683,
   861113: 1172,
   1172: 861113,
   867449: 2151,
   1173: 1443,
   867567: 1333,
   1174: 2346389,
   874193: 2011465,
   1175: 1860835,
   876181: 2638829,
   1176: 1352199,
   882717: 1126257,
   1177: 1097,
   892115: 1223495,
   1178: 1916,
   894473: 1280,
   1179: 1976,
   900463: 1385831,
   1180: 1304,
   908663: 1552387,
   1181: 1199,
   916551: 2480385,
   1182: 1570485,
   924575: 2227047,
   1183: 587125,
   948417: 1866,
   1184: 1261,
   957155: 1631841,
   1185: 1247,
   959621: 2718999,
   1186: 589881,
   962359: 1230191,
   1187: 1394211,
   964713: 1318305,
   1188: 1125,
   981251: 1575841,
   1189: 1152,
   981685: 1110,
   1190: 726431,
   982323: 2461289,
   1191: 1355,
   983511: 1532255,
   1192: 1196,
   985963: 1328,
   1193: 1831603,
   999415: 1519,
   1194: 1348777,
   1000407: 2283455,
   1195: 1126,
   1001637: 731629,
   1196: 1192,
   1002587: 1857323,
   1197: 1211,
   1004329: 1198,
   1198: 1004329,
   1011851: 1111,
   1199: 1181,
   1033007: 1978449,
   1200: 2352995,
   1049013: 1214,
   1201: 1232,
   1059357: 1274,
   1202: 1374925,
   1070983: 1888,
   1203: 2681949,
   1073943: 726183,
   1204: 1377765,
   1077443: 1428,
   1205: 2068,
   1080635: 1284217,
   1206: 730391,
   1088237: 1207,
   1207: 1088237,
   1097625: 1123,
   1208: 1224,
   1100227: 2021701,
   1209: 1718337,
   1101051: 815831,
   1210: 2583199,
   1105539: 1181279,
   1211: 1197,
   1107709: 1314,
   1212: 2018973,
   1115473: 2283143,
   1213: 2761753,
   1116971: 2630463,
   1214: 1049013,
   1119259: 1137,
   1215: 1336987,
   1123417: 784141,
   1216: 1118,
   1126257: 882717,
   1217: 1239,
   1127787: 1725,
   1218: 1707,
   1129859: 2239179,
   1219: 1814173,
   1130879: 821909,
   1220: 1339,
   1140491: 1638359,
   1221: 1272,
   1141793: 1349167,
   1222: 1246869,
   1145895: 1417,
   1223: 1131,
   1150579: 1365,
   1224: 1208,
   1150713: 2136629,
   1225: 1746431,
   1151149: 1495,
   1226: 1283,
   1152229: 1230459,
   1227: 1856,
   1156893: 1399,
   1228: 1301087,
   1157187: 2699851,
   1229: 2845837,
   1161211: 662355,
   1230: 1415,
   1163737: 1231,
   1231: 1163737,
   1167085: 1424,
   1232: 1201,
   1168011: 1527623,
   1233: 1391,
   1170887: 164061,
   1234: 1136,
   1172171: 2644371,
   1235: 1846375,
   1177155: 651259,
   1236: 783399,
   1179591: 1763355,
   1237: 806423,
   1181125: 1433383,
   1238: 1282581,
   1181275: 1371445,
   1239: 1217,
   1181279: 1105539,
   1240: 1808717,
   1182387: 1303,
   1241: 1841,
   1183381: 1293,
   1242: 1292,
   1193135: 2436775,
   1243: 2458037,
   1193613: 1410,
   1244: 1378,
   1196757: 1159,
   1245: 1395,
   1203831: 220655,
   1246: 1289,
   1214063: 1790,
   1247: 1185,
   1218979: 2175,
   1248: 844865,
   1221467: 1143,
   1249: 1358,
   1222679: 1403,
   1250: 1503,
   1223495: 892115,
   1251: 2851445,
   1224777: 1774,
   1252: 1926463,
   1225799: 1791377,
   1253: 1767375,
   1226941: 2738737,
   1254: 1369,
   1230191: 962359,
   1255: 2026,
   1230459: 1152229,
   1256: 782789,
   1234029: 1801065,
   1257: 1157,
   1234663: 1268,
   1258: 1592,
   1236545: 1961301,
   1259: 1885,
   1238075: 2690979,
   1260: 2871939,
   1239479: 2014807,
   1261: 1184,
   1243163: 2695851,
   1262: 2402977,
   1244455: 1349877,
   1263: 1150,
   1246677: 2210355,
   1264: 1315,
   1246869: 1222,
   1265: 1739,
   1250651: 1305,
   1266: 1337,
   1251293: 1404,
   1267: 1140,
   1253615: 1519185,
   1268: 1234663,
   1256141: 1276,
   1269: 731249,
   1258803: 1382361,
   1270: 1869479,
   1262965: 1572839,
   1271: 1350,
   1267611: 1718309,
   1272: 1221,
   1269605: 1396,
   1273: 2094,
   1271293: 1482,
   1274: 1059357,
   1272909: 1120,
   1275: 1285,
   1276051: 1425299,
   1276: 1256141,
   1276181: 646157,
   1277: 728327,
   1282581: 1238,
   1278: 2161,
   1283019: 608697,
   1279: 1551703,
   1284217: 1080635,
   1280: 894473,
   1287847: 1166,
   1281: 1752,
   1300409: 1933,
   1282: 1156,
   1301087: 1228,
   1283: 1226,
   1311313: 1474,
   1284: 1582,
   1318305: 964713,
   1285: 1275,
   1323365: 769207,
   1286: 1736313,
   1325417: 1384,
   1287: 1372507,
   1333997: 1374661,
   1288: 1563,
   1335167: 2116717,
   1289: 1246,
   1336987: 1215,
   1290: 1309,
   1338065: 2673921,
   1291: 1299,
   1344091: 1850681,
   1292: 1242,
   1348777: 1194,
   1293: 1183381,
   1348915: 1405,
   1294: 1860,
   1349167: 1141793,
   1295: 854869,
   1349877: 1244455,
   1296: 1903389,
   1352199: 1176,
   1297: 1595,
   1371445: 1181275,
   1298: 1750,
   1372419: 2025455,
   1299: 1291,
   1372507: 1287,
   1300: 1423457,
   1373785: 2838801,
   1301: 1877643,
   1374091: 653633,
   1302: 1313,
   1374661: 1333997,
   1303: 1182387,
   1374845: 1334,
   1304: 1180,
   1374925: 1202,
   1305: 1250651,
   1376491: 1422915,
   1306: 1437,
   1377765: 1204,
   1307: 1122,
   1378293: 2271349,
   1308: 2846681,
   1382361: 1258803,
   1309: 1290,
   1385783: 1400,
   1310: 1397,
   1385831: 900463,
   1311: 2227745,
   1394211: 1187,
   1312: 1660265,
   1407429: 1712857,
   1313: 1302,
   1409151: 1323,
   1314: 1107709,
   1411245: 1500,
   1315: 1264,
   1418329: 811175,
   1316: 1891491,
   1420887: 1317,
   1317: 1420887,
   1421987: 1335,
   1318: 1349,
   1422915: 1376491,
   1319: 1550311,
   1423457: 1300,
   1320: 1731569,
   1425299: 1276051,
   1321: 1423,
   1425571: 1465133,
   1322: 1505847,
   1430863: 2083123,
   1323: 1409151,
   1433383: 1181125,
   1324: 1729,
   1435195: 1338,
   1325: 1966321,
   1437339: 2623489,
   1326: 1822,
   1437691: 2809787,
   1327: 1668,
   1439067: 1475,
   1328: 985963,
   1439945: 698117,
   1329: 1631303,
   1441973: 1330,
   1330: 1441973,
   1445543: 1910803,
   1331: 1938115,
   1445757: 2344635,
   1332: 1540,
   1449039: 2330119,
   1333: 867567,
   1465133: 1425571,
   1334: 1374845,
   1466315: 2075775,
   1335: 1421987,
   1473855: 704661,
   1336: 1980,
   1478863: 1856209,
   1337: 1266,
   1483013: 1722,
   1338: 1435195,
   1490477: 1890779,
   1339: 1220,
   1492417: 2345215,
   1340: 1910,
   1505847: 1322,
   1341: 1346,
   1508733: 1873259,
   1342: 709075,
   1513043: 2233103,
   1343: 1105,
   1513047: 2339573,
   1344: 2854889,
   1518259: 1979711,
   1345: 2336171,
   1519185: 1253615,
   1346: 1341,
   1524367: 1545,
   1347: 1794225,
   1527623: 1168011,
   1348: 1634959,
   1532255: 983511,
   1349: 1318,
   1536983: 1161,
   1350: 1271,
   1548291: 1958229,
   1351: 2422835,
   1550311: 1319,
   1352: 1506,
   1550999: 2354673,
   1353: 2089701,
   1551703: 1279,
   1354: 1138,
   1552387: 908663,
   1355: 1191,
   1568833: 2423007,
   1356: 1761763,
   1570485: 1182,
   1357: 1407,
   1570547: 1544,
   1358: 1249,
   1571537: 1116,
   1359: 1504,
   1572635: 2914073,
   1360: 1151,
   1572839: 1262965,
   1361: 1988099,
   1575841: 981251,
   1362: 1616,
   1587197: 1602,
   1363: 1558,
   1587583: 1100,
   1364: 1560,
   1593839: 1109,
   1365: 1150579,
   1596895: 1786,
   1366: 2578951,
   1597821: 2243015,
   1367: 1996333,
   1604059: 2846721,
   1368: 2066,
   1604541: 1471,
   1369: 1254,
   1607815: 844189,
   1370: 2002441,
   1612001: 1927049,
   1371: 2746583,
   1621115: 1372,
   1372: 1621115,
   1626763: 1763,
   1373: 1696,
   1630669: 1817,
   1374: 1153,
   1631303: 1329,
   1375: 1960543,
   1631841: 957155,
   1376: 1913143,
   1633263: 1380,
   1377: 1793997,
   1634959: 1348,
   1378: 1244,
   1635641: 1682591,
   1379: 1382,
   1638359: 1140491,
   1380: 1633263,
   1638615: 1740471,
   1381: 1164,
   1644835: 1835975,
   1382: 1379,
   1645953: 1572,
   1383: 2040,
   1648699: 212249,
   1384: 1325417,
   1656987: 2015377,
   1385: 1440,
   1658485: 2132,
   1386: 1740893,
   1660265: 1312,
   1387: 1902987,
   1665693: 1815571,
   1388: 1982563,
   1673269: 1688409,
   1389: 1455,
   1677291: 2746227,
   1390: 2346013,
   1678835: 1726295,
   1391: 1233,
   1679339: 1826,
   1392: 1730087,
   1682355: 1816407,
   1393: 2942729,
   1682591: 1635641,
   1394: 2305135,
   1688409: 1673269,
   1395: 1245,
   1689267: 1541,
   1396: 1269605,
   1691673: 1929653,
   1397: 1310,
   1692049: 1099,
   1398: 1534,
   1692681: 621641,
   1399: 1156893,
   1692763: 1470,
   1400: 1385783,
   1697675: 291177,
   1401: 1888381,
   1698683: 2430941,
   1402: 1615,
   1707755: 1414,
   1403: 1222679,
   1712517: 727775,
   1404: 1251293,
   1712857: 1407429,
   1405: 1348915,
   1718309: 1267611,
   1406: 1625,
   1718337: 1209,
   1407: 1357,
   1723297: 1487,
   1408: 1760991,
   1726295: 1678835,
   1409: 1786749,
   1727435: 2064,
   1410: 1193613,
   1730087: 1392,
   1411: 2131,
   1731569: 1320,
   1412: 1643,
   1734941: 2729559,
   1413: 428211,
   1736313: 1286,
   1414: 1707755,
   1738505: 1149,
   1415: 1230,
   1738617: 1920877,
   1416: 657869,
   1739387: 1719,
   1417: 1145895,
   1740471: 1638615,
   1418: 1901639,
   1740893: 1386,
   1419: 1433,
   1746431: 1225,
   1420: 1709,
   1754559: 1761743,
   1421: 1430,
   1756667: 2086775,
   1422: 1509,
   1757463: 2054791,
   1423: 1321,
   1757623: 2445857,
   1424: 1167085,
   1757785: 1762955,
   1425: 1623,
   1758551: 2113,
   1426: 2325909,
   1760107: 2118341,
   1427: 1860057,
   1760991: 1408,
   1428: 1077443,
   1761371: 1976539,
   1429: 1588,
   1761743: 1754559,
   1430: 1421,
   1761763: 1356,
   1431: 1686,
   1762955: 1757785,
   1432: 1543,
   1763355: 1179591,
   1433: 1419,
   1765719: 1792531,
   1434: 1812,
   1765905: 1165,
   1435: 1639,
   1767375: 1253,
   1436: 1914889,
   1768233: 2882059,
   1437: 1306,
   1768829: 783679,
   1438: 1446,
   1770495: 1889,
   1439: 1484,
   1774405: 1597,
   1440: 1385,
   1780607: 1167,
   1441: 2283987,
   1785191: 1926353,
   1442: 1115,
   1785549: 1806971,
   1443: 1173,
   1786077: 2237691,
   1444: 1538,
   1786675: 2212677,
   1445: 1608,
   1786749: 1409,
   1446: 1438,
   1788911: 2181,
   1447: 1605,
   1788973: 1496,
   1448: 1467,
   1790879: 1851,
   1449: 1479,
   1791149: 1912917,
   1450: 2929957,
   1791377: 1225799,
   1451: 1480,
   1791495: 1858747,
   1452: 1533,
   1792531: 1765719,
   1453: 1507,
   1793997: 1377,
   1454: 2292541,
   1794225: 1347,
   1455: 1389,
   1794231: 1825,
   1456: 1635,
   1795553: 1711,
   1457: 1828001,
   1801065: 1234029,
   1458: 1525,
   1806971: 1785549,
   1459: 1771,
   1807189: 2125,
   1460: 2089,
   1808717: 1240,
   1461: 1951,
   1809325: 637531,
   1462: 2014647,
   1812501: 2678879,
   1463: 1680,
   1814173: 1219,
   1464: 2256865,
   1815129: 1106,
   1465: 1870403,
   1815571: 1665693,
   1466: 1691,
   1816407: 1682355,
   1467: 1448,
   1823887: 1468,
   1468: 1823887,
   1824065: 1518,
   1469: 2141839,
   1825867: 2595771,
   1470: 1692763,
   1825937: 1553,
   1471: 1604541,
   1828001: 1457,
   1472: 2905009,
   1828311: 846827,
   1473: 2093615,
   1831591: 2756929,
   1474: 1311313,
   1831603: 1193,
   1475: 1439067,
   1834561: 1912151,
   1476: 1522,
   1835975: 1644835,
   1477: 1571,
   1844841: 1932,
   1478: 1114,
   1845081: 1517,
   1479: 1449,
   1846375: 1235,
   1480: 1451,
   1848983: 2311713,
   1481: 1794,
   1850681: 1344091,
   1482: 1271293,
   1850929: 2761561,
   1483: 1912339,
   1851301: 1870439,
   1484: 1439,
   1851973: 2074241,
   1485: 1614,
   1852051: 2309889,
   1486: 2174237,
   1854475: 1919359,
   1487: 1723297,
   1856209: 1478863,
   1488: 1497,
   1857323: 1002587,
   1489: 1524,
   1858747: 1791495,
   1490: 2172,
   1860057: 1427,
   1491: 2070,
   1860737: 2478473,
   1492: 1772,
   1860835: 1175,
   1493: 1879,
   1861177: 1657,
   1494: 634833,
   1869479: 1270,
   1495: 1151149,
   1870403: 1465,
   1496: 1788973,
   1870439: 1851301,
   1497: 1488,
   1873259: 1508733,
   1498: 1802,
   1874783: 2231079,
   1499: 2688009,
   1877643: 1301,
   1500: 1411245,
   1879743: 2062,
   1501: 1170,
   1883037: 2289087,
   1502: 1944,
   1884543: 632253,
   1503: 1250,
   1885139: 2179,
   1504: 1359,
   1888381: 1401,
   1505: 2052461,
   1890779: 1490477,
   1506: 1352,
   1891491: 1316,
   1507: 1453,
   1898367: 1915085,
   1508: 2749859,
   1899449: 2163859,
   1509: 1422,
   1899645: 1665,
   1510: 1869,
   1900621: 2264059,
   1511: 2025,
   1900951: 1555,
   1512: 2249677,
   1900989: 2810979,
   1513: 2146,
   1901639: 1418,
   1514: 1967,
   1902067: 2331865,
   1515: 2036,
   1902987: 1387,
   1516: 2280829,
   1903389: 1296,
   1517: 1845081,
   1908725: 2177193,
   1518: 1824065,
   1910719: 2310793,
   1519: 999415,
   1910803: 1445543,
   1520: 1650,
   1911165: 2705963,
   1521: 1938201,
   1912151: 1834561,
   1522: 1476,
   1912339: 1483,
   1523: 2075,
   1912879: 2211773,
   1524: 1489,
   1912917: 1791149,
   1525: 1458,
   1913143: 1376,
   1526: 1843,
   1914313: 2515925,
   1527: 1586,
   1914889: 1436,
   1528: 2169639,
   1915085: 1898367,
   1529: 2020,
   1915405: 2071775,
   1530: 1575,
   1916039: 2317233,
   1531: 1617,
   1919359: 1854475,
   1532: 1682,
   1920851: 2412127,
   1533: 1452,
   1920877: 1738617,
   1534: 1398,
   1924649: 1935509,
   1535: 1556,
   1925505: 1928207,
   1536: 2926595,
   1926181: 2473663,
   1537: 2027,
   1926353: 1785191,
   1538: 1444,
   1926463: 1252,
   1539: 2700763,
   1927049: 1612001,
   1540: 1332,
   1927273: 813467,
   1541: 1689267,
   1928207: 1925505,
   1542: 1773,
   1929653: 1691673,
   1543: 1432,
   1930585: 1576,
   1544: 1570547,
   1932837: 1814,
   1545: 1524367,
   1935509: 1924649,
   1546: 1946,
   1935873: 1946509,
   1547: 2267513,
   1938115: 1331,
   1548: 1959,
   1938201: 1521,
   1549: 2084273,
   1940691: 2259031,
   1550: 1934,
   1942391: 1551,
   1551: 1942391,
   1943525: 1836,
   1552: 2750953,
   1943633: 2587161,
   1553: 1825937,
   1946509: 1935873,
   1554: 2424537,
   1947341: 1601,
   1555: 1900951,
   1952879: 1689,
   1556: 1535,
   1954581: 2804089,
   1557: 1119,
   1956243: 2177139,
   1558: 1363,
   1958229: 1548291,
   1559: 2899293,
   1958605: 1978889,
   1560: 1364,
   1960543: 1375,
   1561: 1699,
   1961301: 1236545,
   1562: 2139,
   1966321: 1325,
   1563: 1288,
   1966851: 775045,
   1564: 2014,
   1969389: 1664,
   1565: 1656,
   1970615: 2336267,
   1566: 1730,
   1974537: 2765841,
   1567: 2564805,
   1974861: 1991077,
   1568: 1770,
   1975919: 2152,
   1569: 2014211,
   1976539: 1761371,
   1570: 1618,
   1976605: 1649,
   1571: 1477,
   1976731: 1629,
   1572: 1645953,
   1977353: 2158,
   1573: 2629823,
   1978449: 1033007,
   1574: 1968,
   1978599: 1726,
   1575: 1530,
   1978889: 1958605,
   1576: 1930585,
   1979711: 1518259,
   1577: 1789,
   1979935: 792153,
   1578: 1993993,
   1980005: 2417997,
   1579: 2562183,
   1982563: 1388,
   1580: 1744,
   1984375: 2783229,
   1581: 1784,
   1984819: 2646953,
   1582: 1284,
   1988099: 1361,
   1583: 1645,
   1988745: 2782309,
   1584: 1600,
   1989553: 2226649,
   1585: 1705,
   1991077: 1974861,
   1586: 1527,
   1993993: 1578,
   1587: 2019,
   1994249: 2142051,
   1588: 1429,
   1996333: 1367,
   1589: 1746,
   2002441: 1370,
   1590: 2004,
   2011465: 874193,
   1591: 2684117,
   2013981: 1849,
   1592: 1258,
   2014211: 1569,
   1593: 1710,
   2014573: 1685,
   1594: 2189,
   2014647: 1462,
   1595: 1297,
   2014793: 1638,
   1596: 2161255,
   2014807: 1239479,
   1597: 1774405,
   2015057: 755761,
   1598: 1900,
   2015377: 1656987,
   1599: 2207753,
   2016911: 2293405,
   1600: 1584,
   2017035: 2111275,
   1601: 1947341,
   2017641: 859445,
   1602: 1587197,
   2018973: 1212,
   1603: 1701,
   2019003: 2961175,
   1604: 2590343,
   2020911: 1778,
   1605: 1447,
   2021701: 1100227,
   1606: 1619,
   2025287: 2485757,
   1607: 1857,
   2025455: 1372419,
   1608: 1445,
   2027307: 1833,
   1609: 2634995,
   2028967: 2430731,
   1610: 2797135,
   2029467: 1721,
   1611: 1847,
   2032401: 2807337,
   1612: 2141677,
   2033003: 2296879,
   1613: 2912135,
   2037837: 2127023,
   1614: 1485,
   2043853: 2689601,
   1615: 1402,
   2044441: 2171997,
   1616: 1362,
   2052461: 1505,
   1617: 1531,
   2053645: 2772065,
   1618: 1570,
   2054225: 1693,
   1619: 1606,
   2054791: 1757463,
   1620: 1754,
   2065091: 2638587,
   1621: 2173,
   2065295: 1622,
   1622: 2065295,
   2065339: 2302963,
   1623: 1425,
   2071775: 1915405,
   1624: 2013,
   2071877: 218351,
   1625: 1406,
   2072325: 1798,
   1626: 2839127,
   2072903: 2489613,
   1627: 1777,
   2074241: 1851973,
   1628: 2135,
   2075775: 1466315,
   1629: 1976731,
   2076645: 2582069,
   1630: 1644,
   2083123: 1430863,
   1631: 1637,
   2084273: 1549,
   1632: 2759363,
   2084743: 1698,
   1633: 1952,
   2086473: 1768,
   1634: 1782,
   2086637: 2077,
   1635: 1456,
   2086775: 1756667,
   1636: 1803,
   2089701: 1353,
   1637: 1631,
   2090459: 2205751,
   1638: 2014793,
   2092107: 2535467,
   1639: 1435,
   2093615: 1473,
   1640: 2100,
   2101713: 1738,
   1641: 2115,
   2102733: 2148015,
   1642: 1688,
   2109385: 2441833,
   1643: 1412,
   2111275: 2017035,
   1644: 1630,
   2116717: 1335167,
   1645: 1583,
   2116829: 1741,
   1646: 2228681,
   2118341: 1760107,
   1647: 1651,
   2125001: 1935,
   1648: 2409689,
   2125135: 2411689,
   1649: 1976605,
   2127023: 2037837,
   1650: 1520,
   2132357: 1655,
   1651: 1647,
   2136629: 1150713,
   1652: 1708,
   2138067: 1993,
   1653: 1747,
   2141677: 1612,
   1654: 1787,
   2141839: 1469,
   1655: 2132357,
   2142051: 1994249,
   1656: 1565,
   2142099: 643505,
   1657: 1861177,
   2143321: 1775,
   1658: 2839815,
   2144713: 1102,
   1659: 1957,
   2144859: 1670,
   1660: 2154415,
   2145315: 2430011,
   1661: 1700,
   2146471: 2737695,
   1662: 1148,
   2146521: 2951535,
   1663: 2155971,
   2148015: 2102733,
   1664: 1969389,
   2148893: 2859539,
   1665: 1899645,
   2150647: 2247423,
   1666: 2456225,
   2150703: 1667,
   1667: 2150703,
   2152283: 1162,
   1668: 1327,
   2153591: 1679,
   1669: 2641849,
   2153601: 1697,
   1670: 2144859,
   2154415: 1660,
   1671: 1925,
   2154761: 2924057,
   1672: 1975,
   2155971: 1663,
   1673: 2183,
   2158831: 2347263,
   1674: 2145,
   2161255: 1596,
   1675: 1813,
   2162891: 2246951,
   1676: 2279685,
   2163859: 1899449,
   1677: 2126,
   2163905: 2405521,
   1678: 2532753,
   2166521: 2184,
   1679: 2153591,
   2167453: 1973,
   1680: 1463,
   2168137: 2630805,
   1681: 1930,
   2169639: 1528,
   1682: 1532,
   2170187: 654589,
   1683: 2071,
   2171661: 2713609,
   1684: 2785857,
   2171997: 2044441,
   1685: 2014573,
   2174237: 1486,
   1686: 1431,
   2177139: 1956243,
   1687: 1793,
   2177193: 1908725,
   1688: 1642,
   2177551: 2710813,
   1689: 1952879,
   2179295: 1907,
   1690: 1988,
   2205751: 2090459,
   1691: 1466,
   2206813: 2337841,
   1692: 2440127,
   2207131: 2120,
   1693: 2054225,
   2207753: 1599,
   1694: 2658325,
   2210355: 1246677,
   1695: 1953,
   2211239: 2452097,
   1696: 1373,
   2211773: 1912879,
   1697: 2153601,
   2211779: 2296453,
   1698: 2084743,
   2212677: 1786675,
   1699: 1561,
   2214763: 2021,
   1700: 1661,
   2215215: 1834,
   1701: 1603,
   2216625: 2493475,
   1702: 1716,
   2218005: 2312343,
   1703: 2083,
   2219299: 2143,
   1704: 2356133,
   2226649: 1989553,
   1705: 1585,
   2227011: 2645965,
   1706: 2177,
   2227047: 924575,
   1707: 1218,
   2227745: 1311,
   1708: 1652,
   2228393: 1107,
   1709: 1420,
   2228681: 1646,
   1710: 1593,
   2231079: 1874783,
   1711: 1795553,
   2232397: 1846,
   1712: 2091,
   2232929: 2788261,
   1713: 2413249,
   2233103: 1513043,
   1714: 1940,
   2235583: 1751,
   1715: 2451027,
   2237055: 1762,
   1716: 1702,
   2237111: 1799,
   1717: 1736,
   2237691: 1786077,
   1718: 1965,
   2239179: 1129859,
   1719: 1739387,
   2239403: 2776643,
   1720: 2034,
   2243015: 1597821,
   1721: 2029467,
   2246951: 2162891,
   1722: 1483013,
   2247021: 1897,
   1723: 1759,
   2247423: 2150647,
   1724: 2130,
   2247535: 2320991,
   1725: 1127787,
   2249131: 2081,
   1726: 1978599,
   2249677: 1512,
   1727: 1824,
   2254741: 1728,
   1728: 2254741,
   2255089: 1887,
   1729: 1324,
   2255167: 2018,
   1730: 1566,
   2256865: 1464,
   1731: 2776317,
   2257097: 1806,
   1732: 2176,
   2259031: 1940691,
   1733: 2420829,
   2261673: 2739675,
   1734: 2270949,
   2264059: 1900621,
   1735: 2352239,
   2265281: 2085,
   1736: 1717,
   2267513: 1547,
   1737: 1776,
   2270949: 1734,
   1738: 2101713,
   2271349: 1378293,
   1739: 1265,
   2275209: 2585983,
   1740: 2700049,
   2279685: 1676,
   1741: 2116829,
   2280829: 1516,
   1742: 1890,
   2283143: 1115473,
   1743: 2550963,
   2283455: 1000407,
   1744: 1580,
   2283987: 1441,
   1745: 2712273,
   2283991: 1104,
   1746: 1589,
   2284105: 2310821,
   1747: 1653,
   2287223: 2551603,
   1748: 2417365,
   2289087: 1883037,
   1749: 2504621,
   2292541: 1454,
   1750: 1298,
   2293405: 2016911,
   1751: 2235583,
   2294693: 2656603,
   1752: 1281,
   2295089: 1823,
   1753: 2191,
   2295955: 1831,
   1754: 1620,
   2296169: 2037,
   1755: 1936,
   2296453: 2211779,
   1756: 2097,
   2296791: 2072,
   1757: 1855,
   2296879: 2033003,
   1758: 2506415,
   2296989: 2606759,
   1759: 1723,
   2299489: 1852,
   1760: 2554257,
   2300461: 1859,
   1761: 1811,
   2301575: 2504749,
   1762: 2237055,
   2302735: 2862779,
   1763: 1626763,
   2302771: 1808,
   1764: 2134,
   2302963: 2065339,
   1765: 2535477,
   2305135: 1394,
   1766: 1913,
   2306549: 2446617,
   1767: 1911,
   2308025: 2473823,
   1768: 2086473,
   2309557: 2654369,
   1769: 2478637,
   2309889: 1852051,
   1770: 1568,
   2310793: 1910719,
   1771: 1459,
   2310821: 2284105,
   1772: 1492,
   2311713: 1848983,
   1773: 1542,
   2312343: 2218005,
   1774: 1224777,
   2312649: 2055,
   1775: 2143321,
   2314949: 2441733,
   1776: 1737,
   2317233: 1916039,
   1777: 1627,
   2318729: 2007,
   1778: 2020911,
   2319615: 2559479,
   1779: 1969,
   2320991: 2247535,
   1780: 2345685,
   2325909: 1426,
   1781: 2731005,
   2327101: 1154,
   1782: 1634,
   2328849: 2565397,
   1783: 2951689,
   2330119: 1449039,
   1784: 1581,
   2331865: 1902067,
   1785: 2332637,
   2332637: 1785,
   1786: 1596895,
   2333153: 2012,
   1787: 1654,
   2334663: 2765643,
   1788: 2088,
   2336171: 1345,
   1789: 1577,
   2336267: 1970615,
   1790: 1214063,
   2337841: 2206813,
   1791: 1821,
   2338101: 2486709,
   1792: 2775795,
   2338241: 2658993,
   1793: 1687,
   2339573: 1513047,
   1794: 1481,
   2340595: 2866293,
   1795: 1962,
   2344635: 1445757,
   1796: 2590127,
   2345215: 1492417,
   1797: 1844,
   2345685: 1780,
   1798: 2072325,
   2346013: 1390,
   1799: 2237111,
   2346389: 1174,
   1800: 2679207,
   2347263: 2158831,
   1801: 2485815,
   2352239: 1735,
   1802: 1498,
   2352995: 1200,
   1803: 1636,
   2354673: 1550999,
   1804: 1867,
   2355501: 2856247,
   1805: 2917189,
   2356133: 1704,
   1806: 2257097,
   2356411: 2751707,
   1807: 1954,
   2361493: 2430261,
   1808: 2302771,
   2362051: 2458389,
   1809: 2887517,
   2363917: 2162,
   1810: 1899,
   2401785: 1876,
   1811: 1761,
   2402977: 1262,
   1812: 1434,
   2405443: 2528583,
   1813: 1675,
   2405521: 2163905,
   1814: 1932837,
   2408297: 2868523,
   1815: 1905,
   2409221: 2964987,
   1816: 1948,
   2409689: 1648,
   1817: 1630669,
   2409887: 2483557,
   1818: 1961,
   2409947: 1819,
   1819: 2409947,
   2411543: 2780785,
   1820: 2774941,
   2411689: 2125135,
   1821: 1791,
   2412127: 1920851,
   1822: 1326,
   2413081: 2587903,
   1823: 2295089,
   2413249: 1713,
   1824: 1727,
   2417365: 1748,
   1825: 1794231,
   2417997: 1980005,
   1826: 1679339,
   2419259: 2533723,
   1827: 1829,
   2419327: 1882,
   1828: 1955,
   2420829: 1733,
   1829: 1827,
   2422835: 1351,
   1830: 1838,
   2423007: 1568833,
   1831: 2295955,
   2424537: 1554,
   1832: 1939,
   2425091: 2078,
   1833: 2027307,
   2427329: 2641193,
   1834: 2215215,
   2429297: 2074,
   1835: 2104,
   2430011: 2145315,
   1836: 1943525,
   2430261: 2361493,
   1837: 2153,
   2430731: 2028967,
   1838: 1830,
   2430941: 1698683,
   1839: 1956,
   2431217: 1982,
   1840: 2054,
   2431691: 2093,
   1841: 1241,
   2436775: 1193135,
   1842: 2808325,
   2436989: 2166,
   1843: 1526,
   2437991: 2588235,
   1844: 1797,
   2440127: 1692,
   1845: 2187,
   2440417: 2484003,
   1846: 2232397,
   2441733: 2314949,
   1847: 1611,
   2441833: 2109385,
   1848: 2614657,
   2442527: 2838251,
   1849: 2013981,
   2443391: 2148,
   1850: 2560735,
   2445857: 1757623,
   1851: 1790879,
   2446617: 2306549,
   1852: 2299489,
   2447595: 2860697,
   1853: 2483431,
   2449593: 2035,
   1854: 2763005,
   2450529: 2855435,
   1855: 1757,
   2450675: 2754001,
   1856: 1227,
   2451027: 1715,
   1857: 1607,
   2452097: 2211239,
   1858: 2060,
   2453103: 2002,
   1859: 2300461,
   2456225: 1666,
   1860: 1294,
   2458037: 1243,
   1861: 2063,
   2458389: 2362051,
   1862: 1929,
   2458979: 1863,
   1863: 2458979,
   2460749: 1864,
   1864: 2460749,
   2461283: 2005,
   1865: 1928,
   2461289: 982323,
   1866: 948417,
   2473663: 1926181,
   1867: 1804,
   2473823: 2308025,
   1868: 2042,
   2478365: 2082,
   1869: 1510,
   2478473: 1860737,
   1870: 2144,
   2478601: 1917,
   1871: 2722303,
   2478637: 1769,
   1872: 2136,
   2480385: 916551,
   1873: 1990,
   2483431: 1853,
   1874: 1983,
   2483557: 2409887,
   1875: 2080,
   2483979: 2714823,
   1876: 2401785,
   2484003: 2440417,
   1877: 2024,
   2485757: 2025287,
   1878: 1926,
   2485815: 1801,
   1879: 1493,
   2486709: 2338101,
   1880: 1999,
   2489613: 2072903,
   1881: 2793267,
   2493475: 2216625,
   1882: 2419327,
   2493771: 2555355,
   1883: 2857579,
   2494315: 2836833,
   1884: 2642961,
   2494345: 2753941,
   1885: 1259,
   2504621: 1749,
   1886: 2595819,
   2504749: 2301575,
   1887: 2255089,
   2506415: 1758,
   1888: 1070983,
   2507307: 2090,
   1889: 1770495,
   2515925: 1914313,
   1890: 1742,
   2519453: 2854455,
   1891: 1994,
   2528583: 2405443,
   1892: 1937,
   2530989: 1893,
   1893: 2530989,
   2531511: 1984,
   1894: 2059,
   2532753: 1678,
   1895: 2645161,
   2533723: 2419259,
   1896: 2180,
   2535467: 2092107,
   1897: 2247021,
   2535477: 1765,
   1898: 1960,
   2537029: 2555743,
   1899: 1810,
   2550963: 1743,
   1900: 1598,
   2551603: 2287223,
   1901: 2816715,
   2554257: 1760,
   1902: 1996,
   2555043: 2028,
   1903: 106983,
   2555355: 2493771,
   1904: 1169,
   2555743: 2537029,
   1905: 1815,
   2559479: 2319615,
   1906: 2003,
   2560735: 1850,
   1907: 2179295,
   2562183: 1579,
   1908: 2022,
   2564393: 2098,
   1909: 2905035,
   2564805: 1567,
   1910: 1340,
   2565397: 2328849,
   1911: 1767,
   2567945: 1966,
   1912: 2702949,
   2573527: 709173,
   1913: 1766,
   2576171: 2822251,
   1914: 2893053,
   2578951: 1366,
   1915: 2735307,
   2582069: 2076645,
   1916: 1178,
   2582177: 2946347,
   1917: 2478601,
   2583199: 1210,
   1918: 2630193,
   2585983: 2275209,
   1919: 2591515,
   2587161: 1943633,
   1920: 2630171,
   2587903: 2413081,
   1921: 1958,
   2588235: 2437991,
   1922: 2717509,
   2588415: 2123,
   1923: 2870711,
   2589615: 2111,
   1924: 2829273,
   2590055: 2897303,
   1925: 1671,
   2590127: 1796,
   1926: 1878,
   2590343: 1604,
   1927: 1978,
   2591515: 1919,
   1928: 1865,
   2595771: 1825867,
   1929: 1862,
   2595819: 1886,
   1930: 1681,
   2602801: 1101,
   1931: 2722059,
   2605123: 2856907,
   1932: 1844841,
   2606759: 2296989,
   1933: 1300409,
   2611269: 2058,
   1934: 1550,
   2613043: 1147,
   1935: 2125001,
   2613965: 2643123,
   1936: 1755,
   2614657: 1848,
   1937: 1892,
   2623489: 1437339,
   1938: 2119,
   2626745: 2700937,
   1939: 1832,
   2629427: 2000,
   1940: 1714,
   2629823: 1573,
   1941: 2096,
   2630171: 1920,
   1942: 2171,
   2630193: 1918,
   1943: 2744495,
   2630463: 1116971,
   1944: 1502,
   2630805: 2168137,
   1945: 2044,
   2634995: 1609,
   1946: 1546,
   2635321: 2785079,
   1947: 2849977,
   2638587: 2065091,
   1948: 1816,
   2638829: 876181,
   1949: 2121,
   2641193: 2427329,
   1950: 2876231,
   2641849: 1669,
   1951: 1461,
   2642425: 1989,
   1952: 1633,
   2642961: 1884,
   1953: 1695,
   2643123: 2613965,
   1954: 1807,
   2644371: 1172171,
   1955: 1828,
   2645161: 1895,
   1956: 1839,
   2645965: 2227011,
   1957: 1659,
   2646953: 1984819,
   1958: 1921,
   2647841: 2127,
   1959: 1548,
   2649847: 2882207,
   1960: 1898,
   2652193: 2031,
   1961: 1818,
   2654369: 2309557,
   1962: 1795,
   2656113: 2108,
   1963: 2807523,
   2656603: 2294693,
   1964: 2669515,
   2658259: 2759605,
   1965: 1718,
   2658325: 1694,
   1966: 2567945,
   2658787: 2103,
   1967: 1514,
   2658923: 2832811,
   1968: 1574,
   2658993: 2338241,
   1969: 1779,
   2664913: 1970,
   1970: 2664913,
   2669515: 1964,
   1971: 2150,
   2673921: 1338065,
   1972: 2687373,
   2678879: 1812501,
   1973: 2167453,
   2679207: 1800,
   1974: 2763861,
   2679531: 2006,
   1975: 1672,
   2681949: 1203,
   1976: 1179,
   2684117: 1591,
   1977: 2943311,
   2686295: 2033,
   1978: 1927,
   2686395: 1979,
   1979: 2686395,
   2686427: 2110,
   1980: 1336,
   2687373: 1972,
   1981: 2140,
   2688009: 1499,
   1982: 2431217,
   2689423: 2893229,
   1983: 1874,
   2689601: 2043853,
   1984: 2531511,
   2690979: 1238075,
   1985: 2817849,
   2693307: 1986,
   1986: 2693307,
   2695851: 1243163,
   1987: 2909343,
   2696891: 2733771,
   1988: 1690,
   2699851: 1157187,
   1989: 2642425,
   2699919: 2808641,
   1990: 1873,
   2700049: 1740,
   1991: 2038,
   2700763: 1539,
   1992: 2829661,
   2700937: 2626745,
   1993: 2138067,
   2702683: 2073,
   1994: 1891,
   2702949: 1912,
   1995: 2883019,
   2705963: 1911165,
   1996: 1902,
   2710813: 2177551,
   1997: 2076,
   2712273: 1745,
   1998: 2102,
   2713609: 2171661,
   1999: 1880,
   2714381: 2138,
   2000: 2629427,
   2714823: 2483979,
   2001: 2842485,
   2717509: 1922,
   2002: 2453103,
   2718999: 959621,
   2003: 1906,
   2722059: 1931,
   2004: 1590,
   2722303: 1871,
   2005: 2461283,
   2724951: 2842843,
   2006: 2679531,
   2729357: 2914149,
   2007: 2318729,
   2729559: 1734941,
   2008: 2777181,
   2731005: 1781,
   2009: 2775521,
   2731849: 2813935,
   2010: 2882861,
   2733771: 2696891,
   2011: 2772737,
   2735307: 1915,
   2012: 2333153,
   2737695: 2146471,
   2013: 1624,
   2738737: 1226941,
   2014: 1564,
   2739675: 2261673,
   2015: 2182,
   2740129: 2893071,
   2016: 2853717,
   2744495: 1943,
   2017: 2051,
   2746227: 1677291,
   2018: 2255167,
   2746583: 1371,
   2019: 1587,
   2749859: 1508,
   2020: 1529,
   2750953: 1552,
   2021: 2214763,
   2751707: 2356411,
   2022: 1908,
   2753941: 2494345,
   2023: 2092,
   2754001: 2450675,
   2024: 1877,
   2756929: 1831591,
   2025: 1511,
   2757001: 2900187,
   2026: 1255,
   2757067: 2760291,
   2027: 1537,
   2759363: 1632,
   2028: 2555043,
   2759605: 2658259,
   2029: 2086,
   2760291: 2757067,
   2030: 2912503,
   2761561: 1850929,
   2031: 2652193,
   2761753: 1213,
   2032: 2164,
   2762099: 2870179,
   2033: 2686295,
   2762767: 2142,
   2034: 1720,
   2763005: 1854,
   2035: 2449593,
   2763861: 1974,
   2036: 1515,
   2765643: 2334663,
   2037: 2296169,
   2765841: 1974537,
   2038: 1991,
   2770011: 2858443,
   2039: 2157,
   2770541: 650237,
   2040: 1383,
   2772065: 2053645,
   2041: 2910943,
   2772737: 2011,
   2042: 1868,
   2774941: 1820,
   2043: 2854953,
   2775521: 2009,
   2044: 1945,
   2775795: 1792,
   2045: 2053,
   2776317: 1731,
   2046: 2095,
   2776643: 2239403,
   2047: 2101,
   2777181: 2008,
   2048: 2826901,
   2777453: 2839181,
   2049: 2155,
   2779043: 2875975,
   2050: 2052,
   2779987: 2821129,
   2051: 2017,
   2780785: 2411543,
   2052: 2050,
   2781391: 729319,
   2053: 2045,
   2782309: 1988745,
   2054: 1840,
   2783229: 1984375,
   2055: 2312649,
   2785079: 2635321,
   2056: 724199,
   2785857: 1684,
   2057: 2163,
   2786595: 2147,
   2058: 2611269,
   2788261: 2232929,
   2059: 1894,
   2789327: 2067,
   2060: 1858,
   2793267: 1881,
   2061: 2159,
   2797135: 1610,
   2062: 1879743,
   2801077: 2167,
   2063: 1861,
   2804089: 1954581,
   2064: 1727435,
   2807193: 2169,
   2065: 2922615,
   2807337: 2032401,
   2066: 1368,
   2807523: 1963,
   2067: 2789327,
   2808325: 1842,
   2068: 1205,
   2808641: 2699919,
   2069: 2156,
   2809787: 1437691,
   2070: 1491,
   2810979: 1900989,
   2071: 1683,
   2813935: 2731849,
   2072: 2296791,
   2814059: 2149,
   2073: 2702683,
   2815685: 2853837,
   2074: 2429297,
   2816715: 1901,
   2075: 1523,
   2817849: 1985,
   2076: 1997,
   2821129: 2779987,
   2077: 2086637,
   2822251: 2576171,
   2078: 2425091,
   2826185: 2079,
   2079: 2826185,
   2826901: 2048,
   2080: 1875,
   2829273: 1924,
   2081: 2249131,
   2829661: 1992,
   2082: 2478365,
   2832811: 2658923,
   2083: 1703,
   2836833: 2494315,
   2084: 2849189,
   2838251: 2442527,
   2085: 2265281,
   2838801: 1373785,
   2086: 2029,
   2839127: 1626,
   2087: 2099,
   2839181: 2777453,
   2088: 1788,
   2839815: 1658,
   2089: 1460,
   2840173: 2898559,
   2090: 2507307,
   2842301: 2190,
   2091: 1712,
   2842485: 2001,
   2092: 2023,
   2842843: 2724951,
   2093: 2431691,
   2843515: 2129,
   2094: 1273,
   2844419: 2845201,
   2095: 2046,
   2845201: 2844419,
   2096: 1941,
   2845837: 1229,
   2097: 1756,
   2845847: 2943301,
   2098: 2564393,
   2846681: 1308,
   2099: 2087,
   2846721: 1604059,
   2100: 1640,
   2849189: 2084,
   2101: 2047,
   2849977: 1947,
   2102: 1998,
   2850177: 2928317,
   2103: 2658787,
   2851445: 1251,
   2104: 1835,
   2853477: 2137,
   2105: 2133,
   2853717: 2016,
   2106: 2141,
   2853837: 2815685,
   2107: 2923211,
   2854455: 2519453,
   2108: 2656113,
   2854677: 2128,
   2109: 1160,
   2854889: 1344,
   2110: 2686427,
   2854953: 2043,
   2111: 2589615,
   2855333: 2932727,
   2112: 2911331,
   2855435: 2450529,
   2113: 1758551,
   2856247: 2355501,
   2114: 2942253,
   2856907: 2605123,
   2115: 1641,
   2857095: 2116,
   2116: 2857095,
   2857579: 1883,
   2117: 1113,
   2858443: 2770011,
   2118: 2910231,
   2859539: 2148893,
   2119: 1938,
   2860697: 2447595,
   2120: 2207131,
   2862779: 2302735,
   2121: 1949,
   2866103: 2950149,
   2122: 2945353,
   2866203: 2889595,
   2123: 2588415,
   2866293: 2340595,
   2124: 2908131,
   2867911: 2933033,
   2125: 1807189,
   2868501: 2930383,
   2126: 1677,
   2868523: 2408297,
   2127: 2647841,
   2870179: 2762099,
   2128: 2854677,
   2870711: 1923,
   2129: 2843515,
   2871939: 1260,
   2130: 1724,
   2875975: 2779043,
   2131: 1411,
   2876231: 1950,
   2132: 1658485,
   2880739: 2174,
   2133: 2105,
   2882059: 1768233,
   2134: 1764,
   2882207: 2649847,
   2135: 1628,
   2882861: 2010,
   2136: 1872,
   2883019: 1995,
   2137: 2853477,
   2887517: 1809,
   2138: 2714381,
   2888089: 2929765,
   2139: 1562,
   2889595: 2866203,
   2140: 1981,
   2893053: 1914,
   2141: 2106,
   2893071: 2740129,
   2142: 2762767,
   2893229: 2689423,
   2143: 2219299,
   2897303: 2590055,
   2144: 1870,
   2898559: 2840173,
   2145: 1674,
   2899293: 1559,
   2146: 1513,
   2900187: 2757001,
   2147: 2786595,
   2905009: 1472,
   2148: 2443391,
   2905035: 1909,
   2149: 2814059,
   2908131: 2124,
   2150: 1971,
   2909343: 1987,
   2151: 867449,
   2910231: 2118,
   2152: 1975919,
   2910943: 2041,
   2153: 1837,
   2911331: 2112,
   2154: 2185,
   2912135: 1613,
   2155: 2049,
   2912503: 2030,
   2156: 2069,
   2914073: 1572635,
   2157: 2039,
   2914149: 2729357,
   2158: 1977353,
   2917189: 1805,
   2159: 2061,
   2917523: 2160,
   2160: 2917523,
   2922615: 2065,
   2161: 1278,
   2923211: 2107,
   2162: 2363917,
   2924057: 2154761,
   2163: 2057,
   2926595: 1536,
   2164: 2032,
   2926919: 2188,
   2165: 2930255,
   2928317: 2850177,
   2166: 2436989,
   2929765: 2888089,
   2167: 2801077,
   2929957: 1450,
   2168: 814285,
   2930211: 2186,
   2169: 2807193,
   2930255: 2165,
   2170: 2948891,
   2930383: 2868501,
   2171: 1942,
   2932727: 2855333,
   2172: 1490,
   2933033: 2867911,
   2173: 1621,
   2934313: 2959699,
   2174: 2880739,
   2942253: 2114,
   2175: 1218979,
   2942729: 1393,
   2176: 1732,
   2942743: 2178,
   2177: 1706,
   2943301: 2845847,
   2178: 2942743,
   2943311: 1977,
   2179: 1885139,
   2943697: 2958163,
   2180: 1896,
   2945353: 2122,
   2181: 1788911,
   2946347: 2582177,
   2182: 2015,
   2948891: 2170,
   2183: 1673,
   2950149: 2866103,
   2184: 2166521,
   2951535: 2146521,
   2185: 2154,
   2951689: 1783,
   2186: 2930211,
   2958163: 2943697,
   2187: 1845,
   2959699: 2934313,
   2188: 2926919,
   2961175: 2019003,
   2189: 1594,
   2961203: 1124,
   2190: 2842301,
   2964987: 2409221,
   2191: 1753
}
print(len(match_data))

2192


In [70]:
score_data = {}

for record in all_people_pref_scores:
    opt_match = match_data[record]

    preferences = all_people_pref_scores[record]

    for pref in preferences:
        if pref == opt_match:
            score_data[record] = {
                "p": pref,
                "score": preferences[pref]
            }

print(score_data)

{71609: {'p': 1096, 'score': 10.0}, 1096: {'p': 71609, 'score': 10.0}, 106983: {'p': 1903, 'score': 10.0}, 1097: {'p': 1177, 'score': 10.0}, 164061: {'p': 1170887, 'score': 10.0}, 1098: {'p': 822315, 'score': 10.0}, 212249: {'p': 1648699, 'score': 9.333}, 1099: {'p': 1692049, 'score': 10.0}, 214227: {'p': 646023, 'score': 10.0}, 1100: {'p': 1587583, 'score': 7.923}, 218351: {'p': 2071877, 'score': 10.0}, 1101: {'p': 2602801, 'score': 10.0}, 220655: {'p': 1203831, 'score': 10.0}, 1102: {'p': 2144713, 'score': 9.333}, 291177: {'p': 1697675, 'score': 10.0}, 1103: {'p': 842437, 'score': 10.0}, 369975: {'p': 855297, 'score': 10.0}, 1104: {'p': 2283991, 'score': 10.0}, 428211: {'p': 1413, 'score': 10.0}, 1105: {'p': 1343, 'score': 10.0}, 497203: {'p': 1130, 'score': 10.0}, 1106: {'p': 1815129, 'score': 10.0}, 516823: {'p': 810569, 'score': 9.923}, 1107: {'p': 2228393, 'score': 10.0}, 582849: {'p': 1168, 'score': 10.0}, 1108: {'p': 1129, 'score': 10.0}, 587125: {'p': 1183, 'score': 10.0}, 110

In [71]:
print(all_people_pref_scores[2020][1529])

9.692


In [72]:
one_to_one = {}

for record in match_data:
    id1 = record
    id2 = match_data[record]

    if(id1 not in one_to_one) and (id2 not in one_to_one):
        one_to_one[id1] = id2
        
print(one_to_one)

{71609: 1096, 106983: 1903, 1097: 1177, 164061: 1170887, 1098: 822315, 212249: 1648699, 1099: 1692049, 214227: 646023, 1100: 1587583, 218351: 2071877, 1101: 2602801, 220655: 1203831, 1102: 2144713, 291177: 1697675, 1103: 842437, 369975: 855297, 1104: 2283991, 428211: 1413, 1105: 1343, 497203: 1130, 1106: 1815129, 516823: 810569, 1107: 2228393, 582849: 1168, 1108: 1129, 587125: 1183, 1109: 1593839, 589881: 1186, 1110: 981685, 608697: 1283019, 1111: 1011851, 621641: 1692681, 1112: 1121, 632253: 1884543, 1113: 2117, 634833: 1494, 1114: 1478, 637531: 1809325, 1115: 1442, 643423: 1155, 1116: 1571537, 643505: 2142099, 1117: 1139, 1118: 1216, 646157: 1276181, 1119: 1557, 648669: 703017, 1120: 1272909, 650237: 2770541, 651259: 1177155, 1122: 1307, 651349: 1134, 1123: 1097625, 653633: 1374091, 1124: 2961203, 654589: 2170187, 1125: 1188, 657869: 1416, 1126: 1195, 662355: 1161211, 1127: 817367, 694739: 1128, 698117: 1439945, 704661: 1473855, 1131: 1223, 705457: 1132, 709075: 1342, 1133: 1141, 709

In [73]:
print(len(one_to_one))

1096


In [74]:
indiv_id_arr_final_matches = []
partnership_edges_final_matches = []

id_labels_final_matches = []
hover_labels_final_matches = []
titles_final_matches = []
colours_final_matches = []


for record in one_to_one:
    id1 = record
    id2 = one_to_one[record]
    pair_arr = [id1, id2]

    for person in pair_arr:
        indiv_id_arr_final_matches.append(person)
        id_labels_final_matches.append(str(person))
    
        poli = all_people[person]["pol"]
        eth = all_people[person]["race"]
        age = all_people[person]["age"]
        edu = all_people[person]["edu"]
    
        person_title = " ".join(["Political Party:", str(poli), "\n",
                              "Race / Eth:", str(eth), "\n",
                              "Age:", str(age), "\n",
                              "Education", str(edu), "\n"])
        
        titles_final_matches.append(person_title)
        colours_final_matches.append("#cc6600")
        
    partnership_edges_final_matches.append([int(id1), int(id2), float((score_data[id1]["score"])*2), "Score: " + str(score_data[id1]["score"])])

from pyvis.network import Network

net_final = Network()

# add nodes
net_final.add_nodes(
    indiv_id_arr_final_matches,
    label=id_labels_final_matches,
    title=titles_final_matches,
    color=colours_final_matches
)
for edge in partnership_edges_final_matches:
    net_final.add_edge(edge[0], edge[1], value=edge[2], title=edge[3])


net_final.show('final_matches_graph_with_scorev3.html', notebook=False)  # save visualization in 'graph.html'

final_matches_graph_with_scorev3.html


In [75]:
total_score_v3 = 0

for record in one_to_one:
    total_score_v3 += score_data[record]["score"]

print("Stable Matching Score: ", total_score_v3)

Stable Matching Score:  10734.875000000002
